In [37]:
import polars as pl
import matplotlib.pyplot as plt
import pathlib


In [38]:


ruta = pathlib.Path().parent.resolve()
data = f"{ruta}/data/sapal_salarios_clean.csv"


salarios_sapal =(
    pl.scan_csv(data)
)

df_filtrado = (
    salarios_sapal.select([
        "denominacion_del_cargo",
        "monto_mensual_bruto_de_la_remuneracion_en_tabulador"
    ])
)

df_filtrado.limit(5).collect()


denominacion_del_cargo,monto_mensual_bruto_de_la_remuneracion_en_tabulador
str,f64
"""SECRETARIA""",10416.6
"""PROGRAMADORA ANALISTA ""A""""",16699.0
"""SUP. CATASTRO FISICO INST.""",19532.3
"""AUX. ADMINISTRATIVA REDES""",12696.0
"""EJECUTIVA DEL SISTEMA DE ATENC…",15239.8


In [39]:
df_filtrado.columns

/var/folders/j4/21ypxx3x0019wgj7rqnlm2340000gn/T/ipykernel_33830/1013323216.py:1: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  df_filtrado.columns


['denominacion_del_cargo',
 'monto_mensual_bruto_de_la_remuneracion_en_tabulador']